# Group 6 Project

In [1]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
dataset = load_dataset("mteb/twentynewsgroups-clustering", revision="6125ec4e24fa026cec8a478383ee943acfbd5449")
print(dataset)

/Users/thorkildkappel/Desktop/8. sem/data mining/project/Data_mining_G6/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    test: Dataset({
        features: ['sentences', 'labels'],
        num_rows: 10
    })
})


### Extracting variables:

In [2]:
test = dataset["test"]
texts_sep = list(test["sentences"])
labels_sep = list(test["labels"])

In [3]:
texts = []
for text in texts_sep:
    texts.extend(text)
print("number of texts: ", len(texts))

number of texts:  59545


### Preprocessing

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(texts)

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1831.64it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [5]:
print(embeddings.shape)
print(embeddings[0][0:20])

(59545, 384)
[-0.10949049  0.01752547  0.06833324 -0.05048881  0.01307424 -0.01668744
 -0.02064349  0.03873773 -0.03209922  0.01640702  0.10264194  0.01111447
  0.07978286 -0.01114772 -0.04782145  0.01050701  0.01809986 -0.02259755
  0.08697543  0.01350243]


In [6]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2**6) #64
features = pca.fit_transform(embeddings)

In [7]:
print(features.shape)

(59545, 64)


In [8]:
test_features = features[0:10000]
print(test_features.shape)
print(test_features[9])

(10000, 64)
[-0.03578575 -0.03294897 -0.2135972   0.06858759  0.09796395  0.05993702
  0.15945302  0.24713865 -0.1793817  -0.11295132  0.28994977 -0.08548318
 -0.0951717  -0.10904121 -0.01815638  0.06955443  0.24568954  0.0419164
  0.09513761 -0.06501797  0.01595114  0.09525589 -0.06524196 -0.06739598
  0.04817227 -0.03976769  0.07842833 -0.08184855 -0.04945495 -0.00266839
  0.09293637 -0.09975026  0.01910595 -0.13573623 -0.01881292  0.07177291
  0.11394461  0.07375485  0.04549788  0.13635264 -0.07872261  0.08253118
  0.0105342   0.03884235 -0.0157774  -0.06864639 -0.05302864 -0.00159399
 -0.13450533 -0.04096956  0.14991264 -0.05404078 -0.03791603  0.02689155
  0.0448002   0.08370029 -0.10620257  0.03845929 -0.10670575  0.06824985
 -0.07485369 -0.00414888  0.02452289  0.03260113]


In [22]:
from src.DBScan import DBScan

scanner =  DBScan(radius=0.6, min_dense=10)
clusters = scanner.make_clusters(features)

(59545,)


In [23]:
import numpy as np
print("Number of clusters produced: ", np.unique(clusters).size)
print(len(np.where(clusters==0)))

Number of clusters produced:  611
1


### Using the DBScan elbow method